<a href="https://colab.research.google.com/github/ewilli400/Trump_Tweets/blob/main/Final_Project_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
#Read in criticisms dataframe, drop duplicates, and make indicator variable
insults_df = pd.read_csv("https://github.com/ewilli400/trump/blob/main/trump_insult_tweets_2014_to_2021.csv?raw=true")
insults_df.drop_duplicates(subset="tweet", inplace=True)
insults_df["indicator"] = 1
insults_df.head()

,Unnamed: 0,date,target,insult,tweet,indicator
0,1,2014-10-09,thomas-frieden,fool,"Can you believe this fool, Dr. Thomas Frieden ...",1
2,3,2015-06-16,politicians,all talk and no action,Big time in U.S. today - MAKE AMERICA GREAT AG...,1
3,4,2015-06-24,ben-cardin,It's politicians like Cardin that have destroy...,Politician @SenatorCardin didn't like that I s...,1
4,5,2015-06-24,neil-young,total hypocrite,"For the nonbeliever, here is a photo of @Neily...",1
5,6,2015-06-24,rockin-in-the-free-world,didn't love it,".@Neilyoung’s song, “Rockin’ In The Free World...",1


In [ ]:
#Read in dataframe of all tweets
all_df = pd.read_csv("https://github.com/ewilli400/trump/blob/main/trumptweets.csv?raw=true")
all_df.head()

,id,link,content,date,retweets,favorites,mentions,hashtags,geo
0,1698308935,https://twitter.com/realDonaldTrump/status/169...,Be sure to tune in and watch Donald Trump on L...,2009-05-04 20:54:25,500,868,NaN,NaN,NaN
1,1701461182,https://twitter.com/realDonaldTrump/status/170...,Donald Trump will be appearing on The View tom...,2009-05-05 03:00:10,33,273,NaN,NaN,NaN
2,1737479987,https://twitter.com/realDonaldTrump/status/173...,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 15:38:08,12,18,NaN,NaN,NaN
3,1741160716,https://twitter.com/realDonaldTrump/status/174...,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 22:40:15,11,24,NaN,NaN,NaN
4,1773561338,https://twitter.com/realDonaldTrump/status/177...,"""My persona will never be that of a wallflower...",2009-05-12 16:07:28,1399,1965,NaN,NaN,NaN


In [ ]:
def to_time(c):
  return c[11:]

def to_date(c):
  return c[0:10]

def to_year(c):
  return c[0:4]

In [ ]:
#Merge both dataframes
trump_tweets = all_df.merge(insults_df, how ="left", left_on="content", right_on="tweet")
trump_tweets["indicator"].fillna(0, inplace=True)
trump_tweets.fillna("None", inplace=True)
trump_tweets = trump_tweets.drop(["tweet", "Unnamed: 0", "date_y", "tweet", "geo", "link", "id"], axis = 1)
trump_tweets = trump_tweets.rename(columns={"date_x": "date"})
trump_tweets['time'] = trump_tweets['date'].map(to_time)
trump_tweets['year'] = trump_tweets['date'].map(to_year).astype(int)
trump_tweets.head()

,content,date,retweets,favorites,mentions,hashtags,target,insult,indicator,time,year
0,Be sure to tune in and watch Donald Trump on L...,2009-05-04 20:54:25,500,868,None,None,None,None,0.0,20:54:25,2009
1,Donald Trump will be appearing on The View tom...,2009-05-05 03:00:10,33,273,None,None,None,None,0.0,03:00:10,2009
2,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 15:38:08,12,18,None,None,None,None,0.0,15:38:08,2009
3,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 22:40:15,11,24,None,None,None,None,0.0,22:40:15,2009
4,"""My persona will never be that of a wallflower...",2009-05-12 16:07:28,1399,1965,None,None,None,None,0.0,16:07:28,2009


In [ ]:
from google.colab import files
trump_tweets.to_csv("out.csv",index=False)
files.download("out.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
all_df['time'] = all_df['date'].map(to_time)
all_df['date'] = all_df['date'].map(to_date)
all_df['year'] = all_df['date'].map(to_year).astype(int)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
#Function to convert tweets into a bag of words
def to_counter(df_year):
  docs = df_year.iloc[:,2]

  vec = CountVectorizer(strip_accents='ascii')
  vec.fit(docs)
  tf_sparse_year = vec.transform(docs)

  df_tf = pd.DataFrame(
    tf_sparse_year.todense(),
    columns=vec.get_feature_names())
  
  df_tf_sum = pd.DataFrame(df_tf.iloc[0:].sum())
  df_tf_sum = df_tf_sum.T

  return df_tf_sum

In [ ]:
#Creating separate dataframes for each year
df_2009 = all_df[all_df['year'] == 2009]
df_2010 = all_df[all_df['year'] == 2010]
df_2011 = all_df[all_df['year'] == 2011]
df_2012 = all_df[all_df['year'] == 2012]
df_2013 = all_df[all_df['year'] == 2013]
df_2014 = all_df[all_df['year'] == 2014]
df_2015 = all_df[all_df['year'] == 2015]
df_2016 = all_df[all_df['year'] == 2016]
df_2017 = all_df[all_df['year'] == 2017]
df_2018 = all_df[all_df['year'] == 2018]
df_2019 = all_df[all_df['year'] == 2019]
df_2020 = all_df[all_df['year'] == 2020]

year_dfs = [df_2009, df_2010, df_2011, df_2012, df_2013, df_2014, df_2015, df_2016, df_2017, df_2018, df_2019, df_2020]

In [ ]:
counter_2009 = to_counter(df_2009)

In [ ]:
#Creating bag of words for each year and concatenating to make term-frequency matrix
df_years = counter_2009

for i in range(1, 12):
  df_i = to_counter(year_dfs[i])
  df_years = pd.concat([df_years, df_i])

In [ ]:
df_years = df_years.fillna(0)
df_years.reset_index(inplace=True)
df_years.index = df_years.index + 2009

In [ ]:
df_years_2015on = df_years.drop(labels=[2009, 2010, 2011, 2012, 2013, 2014])
df_years_2015on = df_years_2015on.replace(0, pd.np.nan).dropna(axis=1, how='any').fillna(0).astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


In [ ]:
import numpy as np
df_years = df_years.replace(0, pd.np.nan).dropna(axis=1, how='any').fillna(0).astype(int)
df_years

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


,about,advice,all,along,an,and,are,as,at,be,beautiful,been,being,best,big,bit,book,business,but,by,call,can,check,com,could,day,days,did,donald,every,everyone,first,for,forget,forward,free,from,great,happy,has,...,or,our,out,people,read,say,show,that,the,them,there,they,think,to,today,tomorrow,tonight,top,trump,twitter,usa,ve,very,was,watch,way,we,what,who,will,win,with,wonderful,work,world,www,year,york,you,your
2009,2,1,5,2,2,20,1,1,2,9,1,1,2,1,3,3,7,3,1,3,1,3,3,42,2,4,1,1,42,1,1,2,7,1,1,1,6,3,4,2,...,1,1,4,1,4,1,2,4,28,1,1,2,9,19,3,1,2,3,48,1,1,1,3,3,5,1,2,1,1,4,1,10,1,1,3,7,1,1,10,10
2010,4,1,9,2,7,66,3,5,39,47,8,3,3,9,1,20,1,1,2,7,3,1,7,31,2,6,1,1,10,3,9,1,50,8,6,1,14,29,8,4,...,1,9,9,1,2,1,14,4,148,2,6,2,1,77,6,8,24,1,49,2,8,3,6,12,6,3,13,2,5,22,5,22,2,3,6,23,2,7,7,6
2011,36,2,34,1,23,289,64,31,115,131,4,11,8,18,7,203,45,7,15,38,3,37,14,28,5,11,1,13,15,5,8,21,124,11,6,3,66,56,10,65,...,20,113,37,26,12,5,23,75,672,10,20,42,10,376,57,10,37,1,68,3,6,5,19,39,52,11,117,24,26,95,4,129,3,11,27,8,14,1,60,44
2012,176,5,223,8,172,993,422,185,356,636,27,78,69,78,91,622,22,44,151,230,18,233,11,70,56,44,24,85,79,45,32,59,690,5,27,22,220,413,45,303,...,87,281,145,139,30,38,84,442,2841,64,77,274,75,1722,111,53,77,30,277,22,22,26,146,269,79,41,324,178,172,489,73,402,28,76,74,34,78,32,512,134
2013,262,28,484,8,214,1846,780,317,604,1054,62,117,157,236,245,520,46,127,387,316,29,388,32,185,94,130,20,131,331,70,76,93,1239,16,63,23,261,1005,115,336,...,198,346,211,346,73,100,183,798,3876,156,159,415,341,2514,133,43,118,69,850,116,48,64,328,382,98,115,425,436,267,807,84,574,38,191,137,33,91,64,2394,662
2014,226,29,340,15,203,1290,551,312,497,897,101,117,94,263,136,203,47,143,267,358,34,432,24,673,87,124,19,65,592,71,39,98,1555,13,82,17,273,786,78,269,...,149,366,183,202,65,78,125,606,3703,128,159,233,214,2230,120,36,34,75,1342,264,63,94,186,336,77,114,518,322,219,629,79,478,37,144,159,288,68,58,2237,591
2015,338,13,470,9,211,1618,677,345,687,1089,60,141,114,240,237,71,73,105,306,480,41,518,21,1677,84,98,19,111,971,86,77,109,1732,15,110,30,283,1088,51,369,...,165,392,310,351,54,119,252,733,4062,129,143,307,166,2688,129,98,234,51,2278,752,78,84,250,407,189,136,712,386,333,972,168,605,65,103,90,662,77,34,2208,536
2016,157,3,297,7,103,1194,358,186,378,601,15,106,54,35,168,139,14,31,141,276,26,177,15,1818,32,67,20,70,142,28,46,54,750,19,37,12,212,574,31,243,...,93,262,231,280,16,53,70,462,2313,82,76,218,37,1479,142,119,132,24,547,1067,19,17,202,277,125,79,429,116,194,794,116,380,47,31,43,613,32,54,994,146
2017,102,4,221,12,105,1151,366,154,269,398,24,75,63,22,157,8,11,37,120,165,17,74,7,724,27,80,19,42,30,21,33,76,578,10,41,11,135,478,41,127,...,86,415,115,201,10,25,21,339,2322,50,82,187,17,1393,150,28,29,13,151,628,16,14,174,226,33,49,370,84,149,484,51,407,47,44,48,79,58,10,303,77
2018,202,5,431,37,205,2850,878,318,486,779,36,240,159,57,252,9,53,44,267,408,29,112,3,666,54,106,25,94,67,53,59,121,1189,11,83,26,283,812,59,490,...,233,930,249,428,17,50,31,854,4854,149,255,646,40,2572,174,31,50,25,294,598,22,2,387,536,36,115,761,176,336,839,110,792,59,80,114,150,103,49,520,151


In [ ]:
df_years.to_csv("years.csv",index=False)
files.download("years.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>